## First Look at Gravity Data

#### Import Data Files

In [ ]:
%reset -f

In [ ]:
import pandas as pd
import hvplot.pandas
import numpy as np
import matplotlib.dates as dates
import warnings
warnings.filterwarnings('ignore')
import holoviews as hv
from holoviews import dim, opts
import hvplot.dask
hv.extension('bokeh')
import glob, os
import dask.dataframe as dd

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

### Start a Dask Cluster

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
import datetime
def dateparse (date_string):
    return datetime.datetime.strptime(date_string, '%d-%m-%Y %H:%M:%S')

In [ ]:
#!ls /home/jovyan/data/bravoseis_data/SADO/jan_2019/gravimetro_bruto.proc/

In [ ]:
!head /Users/dsoule/data/bravoseis_data/SADO/jan_2019/gravimetro_bruto.proc/21012019.gravimetro_bruto.proc

In [ ]:
#!head ~/Dropbox/QueensCollege/Research/BransfieldStrait/data/grav/raw/21012019.gravimetro_bruto.raw

In [ ]:
from dask import delayed
import dask.array as dsa

## Read Gravity Files

In [ ]:
%%time
df_grav=dd.read_csv('/Users/dsoule/data/bravoseis_data/SADO/jan_2019/gravimetro_bruto.proc/*.proc', 
               parse_dates=['fecha'], date_parser=dateparse, 
                    dtype = {'fecha': object,'status': np.float64,
                                'gravimetria_bruta': np.float64, 'spring_tension': np.float64,
                                'longitud': np.float64, 'latitud': np.float64,
                                'velocidad': np.float64,'rumbo': np.float64 })
#df.partitions[5].compute()
df_grav=df_grav.set_index("fecha")
del df_grav['fecha_telegrama']
del df_grav['rumbo']
del df_grav['velocidad']
del df_grav['spring_tension']
del df_grav['status']
df_grav.head()

In [ ]:
df_grav.index.head()

In [ ]:
df_grav = df_grav.resample('s').mean().compute()

## Read Bathy Files

In [ ]:
df_bath=dd.read_csv('/Users/dsoule/data/bravoseis_data/SADO/jan_2019/posicion.proc/*.proc', 
               parse_dates=['fecha'], date_parser=dateparse,
               dtype = {'Date': object,'longitud': np.float64,
                                'latitud': np.float64, 'rumbo': np.float64,
                                'velocidad': np.float64, 'profundidad': np.float64,
                                'cog': np.float64,'sog': np.float64 })
#df.partitions[5].compute()
df_bath=df_bath.set_index("fecha")
del df_bath['fecha_telegrama']
del df_bath['rumbo']
del df_bath['velocidad']
df_bath.head()

In [ ]:
df_bath.index.head()

In [ ]:
df_bath = df_bath.resample('s').mean().compute()

In [ ]:
#df_bath.index = pd.to_datetime(df_bath.index.values)

### Merge Dataframes

In [ ]:
test = pd.merge(df_bath, df_grav,how='inner', indicator=True,left_index=True, right_index=True, suffixes=('_B', '_G'))

In [ ]:
#test = dd.merge(df_bath, df_grav,how='inner',right_index=True, left_index=True,suffixes=('_B', '_G')).compute()

In [ ]:
test.head()

In [ ]:
df_gravMerge = test[test['_merge'] == 'both']
del df_gravMerge['_merge']
df_gravMerge['longitud'] = df_gravMerge['longitud_G']
df_gravMerge['latitud'] = df_gravMerge['latitud_G']
del df_gravMerge['longitud_B']
del df_gravMerge['latitud_B']
del df_gravMerge['longitud_G']
del df_gravMerge['latitud_G']
df_gravMerge.head()

In [ ]:
#df_gravMerge.size

### Downsample the data

In [ ]:
df_minuteGrav = pd.DataFrame()
df_minuteGrav['proc_gravity'] = df_gravMerge.gravimetria_bruta.resample('min').mean()
#df_minuteGrav['eotvos'] = df_gravMerge.eotvos.resample('min').mean()
#df_minuteGrav['grav_corr'] = df_gravMerge.gravimetria_bruta.resample('min').mean() + df_gravMerge.eotvos.resample('min').mean()
df_minuteGrav['lon'] = df_gravMerge.longitud.resample('min').mean()
df_minuteGrav['lat'] = df_gravMerge.latitud.resample('min').mean()
df_minuteGrav['sog'] = df_gravMerge.sog.resample('min').mean()
df_minuteGrav['cog'] = df_gravMerge.cog.resample('min').mean()
df_minuteGrav['depth'] = df_gravMerge.profundidad.resample('min').mean()
df_minuteGrav.tail()

In [ ]:
df_minuteGrav.size

In [ ]:
df_minuteGrav2=df_minuteGrav.loc['2019-01-20 00:00:00':'2019-01-24 00:00:00']
df_temp=df_minuteGrav.loc['2019-01-26 21:00:00':'2019-02-05 23:58:00']
df_minuteGrav2=df_minuteGrav2.append(df_temp)

In [ ]:
df_minuteGrav2.hvplot.points('lon', 'lat', 
                      height=500, 
                      color='proc_gravity', 
                      cmap='colorwheel', 
                      size=3, 
                      hover_cols=['depth'], title= 'proc_gravity',
                      fontsize={'title': 16, 'labels': 14, 'xticks': 12, 'yticks': 12})

In [ ]:
#df_minuteGrav2.hvplot.heatmap(x='lon', y='lat', C='proc_gravity', reduce_function=np.mean, colorbar=True)

### Things to notice:
1. The depth signiature is visable
2. Examine crossing paths... there is a directioal dependence to our readings related to ship direction. 
3. Is the difference between these lines just the ETVOS correction or are their other corrections that need to be applied? 
4. Whould you please share the processing stream? 

In [ ]:
df_minuteGrav2.hvplot.points('index', 'proc_gravity', color='proc_gravity',
                             cmap='colorwheel', size=.5,
                             hover_cols=['cog'], title= 'proc_gravity')

In [ ]:
df_minuteGrav2.head(1)

In [ ]:
cond1 = df_minuteGrav2["lat"] < -62.44    
cond2 = df_minuteGrav2["lat"] > -62.45
cond3 = df_minuteGrav2["lon"] > -58.42
cond4 = df_minuteGrav2["lon"] < -58.36

df_minuteGrav3 = df_minuteGrav2[cond1 & cond2 & cond3 & cond4]

In [ ]:
del df_minuteGrav3['eotvos']
del df_minuteGrav3['grav_corr']
df_minuteGrav3.head()

In [ ]:
df_minuteGrav3.hvplot.scatter('lon', 'lat', 
                      height=500, 
                      color='proc_gravity', 
                      cmap='colorwheel', 
                      size=50, 
                      hover_cols=['depth'], title= 'proc_gravity subset').opts(bgcolor= grey)

In [ ]:
df_minuteGrav3.to_csv('proc_gravity_subset.csv') 

## The gravitational constant in SI units :math:`m^3 kg^{-1} s^{-1}`
## GRAVITATIONAL_CONST = 0.00000000006673

## Bouguer Correction
#### The mass of the material between the gravity station and the datum also causes a variation of gravity with elevation (Figure 1). This mass effect causes gravity at higher stations to be greater than at stations with lower elevations and thus partly offsets the Free Air effect. To calculate the effect of this mass, a model of the topography must be constructed and its density must be estimated.

#### The traditional approach is crude but has been proven to be effective. In this approach, each station is assumed to sit on a slab of material that extends to infinity laterally and to the elevation datum vertically (Figure 1). The formula for the gravitational attraction of this infinite slab is derived by employing a volume integral to calculate its mass. The resulting correction is named for the French geodesist Pierre Bouguer:

## Bouguer Correction = BC = 2pgrh, where g is the International gravitational constant, r is the density, and h = (elevation - datum elevation).

#### As discussed below, the need to estimate density for the calculation of the Bouguer correction is a significant source of uncertainty in gravity studies. With Gobs being observed gravity corrected for drift and tides, the Bouguer anomaly (BA) is then defined as:

## BA = Gobs - Gt + FAC - BC

#### If terrain corrections (see below) are not applied, the term simple Bouguer anomaly is used. If they have, the term complete Bouguer anomaly is used. A second order correction to account for the curvature of the Earth is often added to this calculation.

In [ ]:
ellipsoid = get_ellipsoid()
 #Convert latitude to radians
    latitude_rad = np.radians(latitude)
     prime_vertical_radius = ellipsoid.semimajor_axis / np.sqrt(1 - ellipsoid.first_eccentricity ** 2 * np.sin(latitude_rad) ** 2)
        # Instead of computing X and Y, we only comupute the projection on the XY plane:
        # xy_projection = sqrt( X**2 + Y**2 )
 xy_projection = (height + prime_vertical_radius) * np.cos(latitude_rad)
 z_cartesian = (height + (1 - ellipsoid.first_eccentricity ** 2) * prime_vertical_radius) * np.sin(latitude_rad)
 radius = np.sqrt(xy_projection ** 2 + z_cartesian ** 2)
 geocentric_latitude = 180 / np.pi * np.arcsin(z_cartesian / radius)

    return geocentric_latitude, radius
